# Equations of motion for segbot

This notebook makes use of [SymPy](https://docs.sympy.org/), which is a Python library for symbolic mathematics.

In [15]:
import sympy as sym
import numpy as np

Define physical constants, consistent with the URDF file:

In [33]:
# Dimensions of chassis
dx = 0.4
dy = 0.6
dz = 0.8

# Distance between axle and COM of chassis
h = 0.3

# Half-distance between wheels
a = 0.7 / 2

# Mass of chassis
mb = 12.

# MOI of chassis
Jbx = (mb / 12) * (dy**2 + dz**2)
Jby = (mb / 12) * (dx**2 + dz**2)
Jbz = (mb / 12) * (dx**2 + dy**2)

# Radius of each wheel
r = 0.325

# Width of each wheel
hw = 0.075

# Mass of each wheel
mw = 1.2

# MOI of each wheel
Jw = (mw / 2) * r**2
Jwt = (mw / 12) * (3 * r**2 + hw**2)

# Total mass
m = mb + 2 * mw

# Total MOI
Jx = Jbx + 2 * Jwt
Jy = Jby
Jz = Jbz + 2 * Jwt

# Station parameters
station_velocity = -0.5 # <-- FIXME (change the velocity to change gravity)
station_radius = 20.    # <-- radius in meters of inside surface of station

# Acceleration of artifical gravity
g = station_velocity**2 * station_radius

Convert all physical constants to rational numbers:

In [34]:
# Dimensions
h = sym.nsimplify(h)
a = sym.nsimplify(a)
r = sym.nsimplify(r)

# Masses
mb = sym.nsimplify(mb)
mw = sym.nsimplify(mw)
m = sym.nsimplify(m)

# MOIs
Jx = sym.nsimplify(Jx)
Jy = sym.nsimplify(Jy)
Jz = sym.nsimplify(Jz)
Jw = sym.nsimplify(Jw)
Jwt = sym.nsimplify(Jwt)

# Gravity
g = sym.nsimplify(g)

Define variables (with a flag to indicate they are all real numbers — this helps simplify):

In [35]:
(e_l,
 e_h,
 v,
 w,
 theta,
 thetadot,
 tau_R,
 tau_L) = sym.symbols('e_l, e_h, v, w, theta, thetadot, tau_R, tau_L', real=True)

In [54]:
thetadot

thetadot

Compute equations of motion, excluding lateral and heading errors (see [Tuttle, 2014](https://docs.lib.purdue.edu/cgi/viewcontent.cgi?article=1415&context=open_access_theses)):

In [36]:
M = sym.Matrix([[m + 2 * Jw / r**2, 0, mb * h * sym.cos(theta)],
                [0, (Jx + mb * h**2) * sym.sin(theta)**2 + Jz * sym.cos(theta)**2 + (2 * Jw * a**2 / r**2) + 2 * mw * a**2, 0],
                [mb * h * sym.cos(theta), 0, Jy * mb * h**2]])
N = sym.Matrix([[mb * h * (w**2 + thetadot**2) * sym.sin(theta)],
                [-2 * (Jx - Jz + m * h**2) * sym.cos(theta) * sym.sin(theta) * w * thetadot - mb * h * sym.sin(theta) * v * w],
                [(Jx - Jz + mb * h**2) * sym.cos(theta) * sym.sin(theta) * w**2 + mb * g * h * sym.sin(theta)]])
R = sym.Matrix([[1 / r, 1 / r],
                [-a / r, a / r],
                [-1, -1]])
f = sym.simplify(M.inv() * (N + R * sym.Matrix([[tau_L], [tau_R]])))

In [37]:
M

Matrix([
[           78/5,                                                            0, 18*cos(theta)/5],
[              0, 4289*sin(theta)**2/2000 + 1169*cos(theta)**2/2000 + 441/1000,               0],
[18*cos(theta)/5,                                                            0,         108/125]])

In [38]:
N

Matrix([
[                      (18*thetadot**2/5 + 18*w**2/5)*sin(theta)],
[-444*thetadot*w*sin(theta)*cos(theta)/125 - 18*v*w*sin(theta)/5],
[               39*w**2*sin(theta)*cos(theta)/25 + 18*sin(theta)]])

In [39]:
R

Matrix([
[ 40/13, 40/13],
[-14/13, 14/13],
[    -1,    -1]])

In [40]:
f

Matrix([
[        -(1200*tau_L + 1200*tau_R + 1404*(thetadot**2 + w**2)*sin(theta) + 65*(50*tau_L + 50*tau_R - 39*w**2*sin(2*theta) - 900*sin(theta))*cos(theta)/2)/(5850*cos(theta)**2 - 6084)],
[                                                                    32*(-875*tau_L + 875*tau_R - 1443*thetadot*w*sin(2*theta) - 2925*v*w*sin(theta))/(13*(3120*sin(theta)**2 + 2051))],
[5*(4225*tau_L + 4225*tau_R - 6591*w**2*sin(2*theta)/2 + 30*(100*tau_L + 100*tau_R + 117*(thetadot**2 + w**2)*sin(theta))*cos(theta) - 76050*sin(theta))/(702*(25*cos(theta)**2 - 26))]])

Compute full equations of motion:

In [41]:
f = sym.Matrix([[v * sym.sin(e_h)],
                [w],
                [f]])

Display the vector-valued function $f$ for which the equations of motion can be written as

$$\begin{bmatrix} \dot{e}_\text{lateral} \\ \dot{e}_\text{heading} \\ \dot{v} \\ \dot{w} \\ \ddot{\theta} \end{bmatrix} = f(e_\text{lateral}, e_\text{heading}, v, w, \theta, \dot{\theta}, \tau_R, \tau_L)$$

In [42]:
f

Matrix([
[                                                                                                                                                                           v*sin(e_h)],
[                                                                                                                                                                                    w],
[        -(1200*tau_L + 1200*tau_R + 1404*(thetadot**2 + w**2)*sin(theta) + 65*(50*tau_L + 50*tau_R - 39*w**2*sin(2*theta) - 900*sin(theta))*cos(theta)/2)/(5850*cos(theta)**2 - 6084)],
[                                                                    32*(-875*tau_L + 875*tau_R - 1443*thetadot*w*sin(2*theta) - 2925*v*w*sin(theta))/(13*(3120*sin(theta)**2 + 2051))],
[5*(4225*tau_L + 4225*tau_R - 6591*w**2*sin(2*theta)/2 + 30*(100*tau_L + 100*tau_R + 117*(thetadot**2 + w**2)*sin(theta))*cos(theta) - 76050*sin(theta))/(702*(25*cos(theta)**2 - 26))]])

Note that one of these ODEs is second-order — you will have to replace this with a set of two first-order ODEs, as usual.

In [43]:
f[0]

v*sin(e_h)

In [44]:
f[1]

w

In [45]:
f[2]

-(1200*tau_L + 1200*tau_R + 1404*(thetadot**2 + w**2)*sin(theta) + 65*(50*tau_L + 50*tau_R - 39*w**2*sin(2*theta) - 900*sin(theta))*cos(theta)/2)/(5850*cos(theta)**2 - 6084)

In [46]:
f[3]

32*(-875*tau_L + 875*tau_R - 1443*thetadot*w*sin(2*theta) - 2925*v*w*sin(theta))/(13*(3120*sin(theta)**2 + 2051))

In [47]:
f[4]

5*(4225*tau_L + 4225*tau_R - 6591*w**2*sin(2*theta)/2 + 30*(100*tau_L + 100*tau_R + 117*(thetadot**2 + w**2)*sin(theta))*cos(theta) - 76050*sin(theta))/(702*(25*cos(theta)**2 - 26))

In [55]:
phi, phidot = sym.symbols('phi, phidot')

In [57]:
phidot

phidot

In [58]:
fnew = sym.Matrix([[f[0]], [f[1]], [f[2]], [f[3]], [phi], [f[4]]])

fnew, fnew.shape

Matrix([
[                                                                                                                                                                           v*sin(e_h)],
[                                                                                                                                                                                    w],
[        -(1200*tau_L + 1200*tau_R + 1404*(thetadot**2 + w**2)*sin(theta) + 65*(50*tau_L + 50*tau_R - 39*w**2*sin(2*theta) - 900*sin(theta))*cos(theta)/2)/(5850*cos(theta)**2 - 6084)],
[                                                                    32*(-875*tau_L + 875*tau_R - 1443*thetadot*w*sin(2*theta) - 2925*v*w*sin(theta))/(13*(3120*sin(theta)**2 + 2051))],
[                                                                                                                                                                                  phi],
[5*(4225*tau_L + 4225*tau_R - 6591*w**2*sin(2*theta)/2 + 30*(100*t

## Equilibrium Point Finding

In [60]:
taul, taur = sym.symbols('taul, taur')

In [64]:
dotv = (-2400*taul + 2400*taur + 65*(50*taul + 50*taur)) / (11700-12168)
dotw = (32*(-875*taul + 875*taur)) / (13*2051)
dotphi = (5*(8450*taul + 8450*taur+60*(100*taul + 100*taur))) / (1404*(25-26))

dotv_eq = sym.Eq(dotv, 0)
dotw_eq = sym.Eq(dotw, 0)
dotphi_eq = sym.Eq(dotphi, 0)

solved = sym.solve((vdot, wdot, dottheta), (taul, taur))
solved

[]

In [67]:
e_le = 0
e_he = 0 # angle
ve = 0
we = 0
phie = 0 # angle
phidote = 0
tau_Re = 0
tau_Le = 0

## A, B Values

In [80]:
A_num = sym.lambdify((e_l,e_h, v, w, theta, thetadot, tau_L, tau_R), fnew.jacobian([e_l, e_h, v, w, theta, thetadot]))
B_num = sym.lambdify((e_l,e_h, v, w, theta, thetadot, tau_L, tau_R), fnew.jacobian([tau_L, tau_R]))

A = A_num(e_le, e_he, ve, we, phie, phidote, tau_Re, tau_Le)
B = B_num(e_le, e_he, ve, we, phie, phidote, tau_Re, tau_Le)

A = A.astype(float)
B = B.astype(float)

A, B

(array([[   0.        ,    0.        ,    0.        ,    0.        ,
            0.        ,    0.        ],
        [   0.        ,    0.        ,    0.        ,    1.        ,
            0.        ,    0.        ],
        [   0.        ,    0.        ,    0.        ,    0.        ,
         -125.        ,   -0.        ],
        [   0.        ,    0.        ,    0.        ,    0.        ,
            0.        ,    0.        ],
        [   0.        ,    0.        ,    0.        ,    0.        ,
            0.        ,    0.        ],
        [   0.        ,    0.        ,    0.        ,   -0.        ,
          541.66666667,   -0.        ]]),
 array([[  0.        ,   0.        ],
        [  0.        ,   0.        ],
        [ 12.07264957,  12.07264957],
        [ -1.05014439,   1.05014439],
        [  0.        ,   0.        ],
        [-51.46011396, -51.46011396]]))

In [81]:
A.shape, B.shape

((6, 6), (6, 2))

## Finding K Value (A: 5x6, B:5x2), deprecated

In [83]:
## F = A - B@K, K: 2x6 matrix

import random

check = True
k = np.array([[0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0]])
while check:
    a = random.uniform(-15,15)
    b = random.uniform(-15,15)
    c = random.uniform(-15,15)
    d = random.uniform(-15,15)
    e = random.uniform(-15,15)
    f = random.uniform(-15,15)
    g = random.uniform(-15,15)
    h = random.uniform(-15,15)
    i = random.uniform(-15,15)
    j = random.uniform(-15,15)
    l = random.uniform(-15,15)
    m = random.uniform(-15,15)
    
    k = np.array([[a, b, c, d, e, f], [g, h, i, j, l, m]])
    vals = np.linalg.eigvals(A-B@k)
    if (vals[0] < 0 and vals[1] < 0 and vals[2] < 0 and vals[3] < 0):
        check = False
print(k)
print(np.linalg.eigvals(A-B@k))


[[ 12.62708954 -12.18499175   9.96311334  11.51395985   1.00446901
   -8.08037286]
 [ -6.28458123   5.92891486  11.55246637  11.0950538    6.37259325
   -5.50921159]]
[-9.56263427e+02+0.j         -1.18414018e+00+4.11080223j
 -1.18414018e+00-4.11080223j -4.14937402e-13+0.j
  0.00000000e+00+0.j          0.00000000e+00+0.j        ]


## Finding K the Right Way